In [1]:
#Import Pandas

import pandas as pd

In [11]:
#Load the data 

voc_training=pd.read_csv('Voc_training.csv', encoding="latin1", sep=';')
voc_training.head()

,Bundesland,Ausbildungsbereich,Geschlecht,Auszubildende Alter,2018,2017,2016,2015,2014,2013,2012
0,Land Berlin,Industrie und Handel,männlich,15 und jünger,1,3,2,4,2,2,3
1,Land Berlin,Industrie und Handel,männlich,16,335,264,317,315,223,229,233
2,Land Berlin,Industrie und Handel,männlich,17,795,815,786,662,705,803,773
3,Land Berlin,Industrie und Handel,männlich,18,1.504,1.372,1.220,1.216,1.305,1.262,1.251
4,Land Berlin,Industrie und Handel,männlich,19,1.819,1.717,1.692,1.796,1.653,1.678,1.759


In [46]:
#Transforming from wide to long format

voc_training["id"] = voc_training.index

print(voc_training.keys()[0:4])

voc_training_long=pd.wide_to_long(voc_training, stubnames=['20'], 
                                  i='id', 
                                  j='year')
voc_training_long.head(n=10)

#Renaming the '20'column to a meaningful name, 'Anzahl', which means number 

voc_training_long.rename(columns={'20':'Anzahl'}, inplace=True)
voc_training_long.head(n=10)

Index(['Bundesland', 'Ausbildungsbereich', 'Geschlecht',
       'Auszubildende Alter'],
      dtype='object')


,,Auszubildende Alter,Ausbildungsbereich,Geschlecht,Bundesland,Anzahl
id,year,,,,,
0,18,15 und jünger,Industrie und Handel,männlich,Land Berlin,1
1,18,16,Industrie und Handel,männlich,Land Berlin,335
2,18,17,Industrie und Handel,männlich,Land Berlin,795
3,18,18,Industrie und Handel,männlich,Land Berlin,1.504
4,18,19,Industrie und Handel,männlich,Land Berlin,1.819
5,18,20,Industrie und Handel,männlich,Land Berlin,1.945
6,18,21,Industrie und Handel,männlich,Land Berlin,1.805
7,18,22,Industrie und Handel,männlich,Land Berlin,1.378
8,18,23,Industrie und Handel,männlich,Land Berlin,991


This was the first try. 